In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install torch
!pip install matplotlib
!pip install scipy
!pip install scikit-learn
!pip install torch_geometric
!pip install tensorflow

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges, to_undirected
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.signal import stft
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
# 데이터 파일 로드 및 정규화

def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

def load_and_normalize_data(filenames, label):
    data_list = []
    for filename in filenames:
        df = pd.read_csv(filename)
        motor_data_normalized = normalize_data(df[['motor1_x', 'motor1_y', 'motor1_z']])
        sound_data_normalized = normalize_data(df[['sound']])
        data_list.append((motor_data_normalized, sound_data_normalized, label))
    return data_list

In [ ]:
path="drive/MyDrive/Colab Notebooks/SmartFactory/csv/5000hz/300rpm/"

normal_files = [path+"300rpm normal data/stream2024_4_22_23_27.csv"]
fault1_files = [path+"300rpm carriage damage/stream2024_4_23_3_1.csv"]
fault2_files = [path+"300rpm high-speed damage/stream2024_4_23_0_24.csv"]
fault3_files = [path+"300rpm lack of lubrication/stream2024_4_23_2_10.csv"]
fault4_files = [path+"300rpm oxidation and corrosion/stream2024_4_23_1_19.csv"]

data_normal = load_and_normalize_data(normal_files, 0)
data_fault1 = load_and_normalize_data(fault1_files, 1)
data_fault2 = load_and_normalize_data(fault2_files, 2)
data_fault3 = load_and_normalize_data(fault3_files, 3)
data_fault4 = load_and_normalize_data(fault4_files, 4)

all_data = data_normal + data_fault1 + data_fault2 + data_fault3 + data_fault4

In [ ]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

# 데이터 파일 로드
normal_data = pd.read_csv(path+"300rpm normal data/stream2024_4_22_23_27.csv")
carriage_data = pd.read_csv(path+"300rpm carriage damage/stream2024_4_23_3_1.csv")
highspeed_data = pd.read_csv(path+"300rpm high-speed damage/stream2024_4_23_0_24.csv")
lack_data = pd.read_csv(path+"300rpm lack of lubrication/stream2024_4_23_2_10.csv")
corrosion_data = pd.read_csv(path+"300rpm oxidation and corrosion/stream2024_4_23_1_19.csv")

# 데이터 정규화 함수 정의
def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

# 모터 및 소리 데이터 정규화
normal_motor_data_normalized = normalize_data(normal_data[['motor1_x', 'motor1_y', 'motor1_z']])
normal_sound_data_normalized = normalize_data(normal_data[['sound']])
carriage_motor_data_normalized = normalize_data(carriage_data[['motor1_x', 'motor1_y', 'motor1_z']])
carriage_sound_data_normalized = normalize_data(carriage_data[['sound']])
highspeed_motor_data_normalized = normalize_data(highspeed_data[['motor1_x', 'motor1_y', 'motor1_z']])
highspeed_sound_data_normalized = normalize_data(highspeed_data[['sound']])
lack_motor_data_normalized = normalize_data(lack_data[['motor1_x', 'motor1_y', 'motor1_z']])
lack_sound_data_normalized = normalize_data(lack_data[['sound']])
corrosion_motor_data_normalized = normalize_data(corrosion_data[['motor1_x', 'motor1_y', 'motor1_z']])
corrosion_sound_data_normalized = normalize_data(corrosion_data[['sound']])

# 데이터 합치기
def concatenate_data(motor_data, sound_data):
    return np.hstack((motor_data, sound_data))

X_normal = concatenate_data(normal_motor_data_normalized, normal_sound_data_normalized)
X_carriage = concatenate_data(carriage_motor_data_normalized, carriage_sound_data_normalized)
X_highspeed = concatenate_data(highspeed_motor_data_normalized, highspeed_sound_data_normalized)
X_lack = concatenate_data(lack_motor_data_normalized, lack_sound_data_normalized)
X_corrosion = concatenate_data(corrosion_motor_data_normalized, corrosion_sound_data_normalized)

# 레이블 생성
y_normal = np.zeros(X_normal.shape[0])
y_carriage = np.ones(X_carriage.shape[0])
y_highspeed = np.full((X_highspeed.shape[0],), 2)
y_lack = np.full((X_lack.shape[0],), 3)
y_corrosion = np.full((X_corrosion.shape[0],), 4)

# 데이터셋 합치기
X = np.vstack((X_normal, X_carriage, X_highspeed, X_lack, X_corrosion))
y = np.concatenate((y_normal, y_carriage, y_highspeed, y_lack, y_corrosion))

# 레이블을 원-핫 인코딩으로 변환
y = to_categorical(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 형태 확인
timesteps = 1
input_dim = X_train.shape[1]

# 데이터 형태 변환
X_train = X_train.reshape(X_train.shape[0], timesteps, input_dim)
X_test = X_test.reshape(X_test.shape[0], timesteps, input_dim)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, input_dim), activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 얼리스탑 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), callbacks=[early_stopping])


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes,digits=4))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 128)            68096     
                                                                 
 lstm_1 (LSTM)               (None, 1, 64)             49408     
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 132357 (517.02 KB)
Trainable params: 13235

In [ ]:
#LSTM AE

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
import scipy.signal
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Input, LeakyReLU
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import kneighbors_graph



# 데이터 파일 로드
normal_data = pd.read_csv(path+"300rpm normal data/stream2024_4_22_23_27.csv")
carriage_data = pd.read_csv(path+"300rpm carriage damage/stream2024_4_23_3_1.csv")
highspeed_data = pd.read_csv(path+"300rpm high-speed damage/stream2024_4_23_0_24.csv")
lack_data = pd.read_csv(path+"300rpm lack of lubrication/stream2024_4_23_2_10.csv")
corrosion_data = pd.read_csv(path+"300rpm oxidation and corrosion/stream2024_4_23_1_19.csv")

# 데이터 정규화 함수 정의
def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)
def concatenate_data(motor_data, sound_data):
    return np.hstack((motor_data, sound_data))

# 모터 및 소리 데이터 정규화
normal_motor_data_normalized = normalize_data(normal_data[['motor1_x', 'motor1_y', 'motor1_z']])
normal_sound_data_normalized = normalize_data(normal_data[['sound']])
carriage_motor_data_normalized = normalize_data(carriage_data[['motor1_x', 'motor1_y', 'motor1_z']])
carriage_sound_data_normalized = normalize_data(carriage_data[['sound']])
highspeed_motor_data_normalized = normalize_data(highspeed_data[['motor1_x', 'motor1_y', 'motor1_z']])
highspeed_sound_data_normalized = normalize_data(highspeed_data[['sound']])
lack_motor_data_normalized = normalize_data(lack_data[['motor1_x', 'motor1_y', 'motor1_z']])
lack_sound_data_normalized = normalize_data(lack_data[['sound']])
corrosion_motor_data_normalized = normalize_data(corrosion_data[['motor1_x', 'motor1_y', 'motor1_z']])
corrosion_sound_data_normalized = normalize_data(corrosion_data[['sound']])

X_normal = concatenate_data(normal_motor_data_normalized, normal_sound_data_normalized)
X_carriage = concatenate_data(carriage_motor_data_normalized, carriage_sound_data_normalized)
X_highspeed = concatenate_data(highspeed_motor_data_normalized, highspeed_sound_data_normalized)
X_lack = concatenate_data(lack_motor_data_normalized, lack_sound_data_normalized)
X_corrosion = concatenate_data(corrosion_motor_data_normalized, corrosion_sound_data_normalized)


# 레이블 생성
y_normal = np.zeros(X_normal.shape[0])
y_carriage = np.ones(X_carriage.shape[0])
y_highspeed = np.full((X_highspeed.shape[0],), 2)
y_lack = np.full((X_lack.shape[0],), 3)
y_corrosion = np.full((X_corrosion.shape[0],), 4)

# 데이터셋 합치기
X = np.concatenate((X_normal, X_carriage, X_highspeed, X_lack, X_corrosion), axis=0)
y = np.concatenate((y_normal, y_carriage, y_highspeed, y_lack, y_corrosion), axis=0)
# 레이블을 원-핫 인코딩으로 변환
y = to_categorical(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 형태 확인
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# 데이터 형태 변환
X_train = X_train.reshape(X_train.shape[0],-1, X_train.shape[1])  # (샘플 수, 시간, 특징 수)
X_test = X_test.reshape(X_test.shape[0], -1,X_test.shape[1])
print("Train data shape after reshape:", -1,X_train.shape)
print("Test data shape after reshape:", -1,X_test.shape)

# LSTM Autoencoder 모델 정의
def create_lstm_ae(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(32, return_sequences=False))
    model.add(RepeatVector(input_shape[0]))
    model.add(LSTM(32, return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(TimeDistributed(Dense(input_shape[1])))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # (시간, 특징 수)
model = create_lstm_ae(input_shape)

model.compile(optimizer='adam', loss='mse')

# 얼리스탑 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 모델 훈련
print("Training LSTM Autoencoder model...")
history = model.fit(X_train, X_train, epochs=1, batch_size=256, validation_data=(X_test, X_test), callbacks=[early_stopping])


# LSTM Autoencoder에서 인코딩된 특징 추출
encoder = Sequential()
encoder.add(model.layers[0])
encoder.add(model.layers[1])
encoder.compile(optimizer='adam', loss='mse')

X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)


classifier = Sequential()
classifier.add(Dense(64, input_shape=(X_train_encoded.shape[1],)))
classifier.add(LeakyReLU(alpha=0.01))  # Leaky ReLU 활성화 함수 사용
classifier.add(Dropout(0.5))
classifier.add(Dense(32))
classifier.add(LeakyReLU(alpha=0.01))  # Leaky ReLU 활성화 함수 사용
classifier.add(Dropout(0.2))
classifier.add(Dense(y_train.shape[1], activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 다중 클래스 분류 모델 훈련
print("Training classifier model...")
history = classifier.fit(X_train_encoded, y_train, epochs=1, batch_size=256, validation_data=(X_test_encoded, y_test), callbacks=[early_stopping])

# 테스트 데이터에 대한 예측 및 평가
y_test_pred = classifier.predict(X_test_encoded)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

print("Accuracy on test set:", accuracy_score(y_test_true_classes, y_test_pred_classes))
print(classification_report(y_test_true_classes, y_test_pred_classes, target_names=['Normal', 'Carriage Damage', 'High-Speed Damage', 'Lack of Lubrication', 'Oxidation and Corrosion'],digits=4))

Train data shape: (1200000, 4)
Test data shape: (300000, 4)
Train data shape after reshape: -1 (1200000, 1, 4)
Test data shape after reshape: -1 (300000, 1, 4)
Training LSTM Autoencoder model...
9375/9375 [==============================] - 21s 2ms/step
Training classifier model...
9375/9375 [==============================] - 16s 2ms/step
Accuracy on test set: 0.25787333333333334
                         precision    recall  f1-score   support

                 Normal     0.2551    0.2783    0.2662     59643
        Carriage Damage     0.2132    0.0844    0.1209     59988
      High-Speed Damage     0.2140    0.1249    0.1577     60001
    Lack of Lubrication     0.2471    0.3624    0.2938     60196
Oxidation and Corrosion     0.3003    0.4386    0.3565     60172

               accuracy                         0.2579    300000
              macro avg     0.2459    0.2577    0.2390    300000
           weighted avg     0.2460    0.2579    0.2391    300000



In [ ]:

#transformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.signal import stft
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# 데이터 파일 로드
normal_data = pd.read_csv(path+"300rpm normal data/stream2024_4_22_23_27.csv")
carriage_data = pd.read_csv(path+"300rpm carriage damage/stream2024_4_23_3_1.csv")
highspeed_data = pd.read_csv(path+"300rpm high-speed damage/stream2024_4_23_0_24.csv")
lack_data = pd.read_csv(path+"300rpm lack of lubrication/stream2024_4_23_2_10.csv")
corrosion_data = pd.read_csv(path+"300rpm oxidation and corrosion/stream2024_4_23_1_19.csv")

# 데이터 정규화 함수 정의
def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)
def concatenate_data(motor_data, sound_data):
    return np.hstack((motor_data, sound_data))

# 모터 및 소리 데이터 정규화
normal_motor_data_normalized = normalize_data(normal_data[['motor1_x', 'motor1_y', 'motor1_z']])
normal_sound_data_normalized = normalize_data(normal_data[['sound']])
carriage_motor_data_normalized = normalize_data(carriage_data[['motor1_x', 'motor1_y', 'motor1_z']])
carriage_sound_data_normalized = normalize_data(carriage_data[['sound']])
highspeed_motor_data_normalized = normalize_data(highspeed_data[['motor1_x', 'motor1_y', 'motor1_z']])
highspeed_sound_data_normalized = normalize_data(highspeed_data[['sound']])
lack_motor_data_normalized = normalize_data(lack_data[['motor1_x', 'motor1_y', 'motor1_z']])
lack_sound_data_normalized = normalize_data(lack_data[['sound']])
corrosion_motor_data_normalized = normalize_data(corrosion_data[['motor1_x', 'motor1_y', 'motor1_z']])
corrosion_sound_data_normalized = normalize_data(corrosion_data[['sound']])

X_normal = concatenate_data(normal_motor_data_normalized, normal_sound_data_normalized)
X_carriage = concatenate_data(carriage_motor_data_normalized, carriage_sound_data_normalized)
X_highspeed = concatenate_data(highspeed_motor_data_normalized, highspeed_sound_data_normalized)
X_lack = concatenate_data(lack_motor_data_normalized, lack_sound_data_normalized)
X_corrosion = concatenate_data(corrosion_motor_data_normalized, corrosion_sound_data_normalized)

# 레이블 생성
y_normal = np.zeros(X_normal.shape[0])
y_carriage = np.ones(X_carriage.shape[0])
y_highspeed = np.full((X_highspeed.shape[0],), 2)
y_lack = np.full((X_lack.shape[0],), 3)
y_corrosion = np.full((X_corrosion.shape[0],), 4)

# 데이터셋 합치기
X = np.concatenate((X_normal, X_carriage, X_highspeed, X_lack, X_corrosion), axis=0)
y = np.concatenate((y_normal, y_carriage, y_highspeed, y_lack, y_corrosion), axis=0)


# 데이터 분할 (Train: 80%, Validation: 10%, Test: 10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Transformer 모델 정의
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_encoder_layers=3, dim_feedforward=128, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout),
            num_layers=num_encoder_layers
        )
        self.fc = nn.Linear(d_model, num_classes)
        self.input_dim = input_dim
        self.d_model = d_model
        self.linear = nn.Linear(input_dim, d_model)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        x = self.linear(x)  # (batch_size, seq_len, d_model)
        x = x.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        x = self.encoder(x)  # (seq_len, batch_size, d_model)
        x = x.mean(dim=0)  # (batch_size, d_model)
        x = self.fc(x)  # (batch_size, num_classes)
        return x

# 하이퍼파라미터 설정
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))

model = TimeSeriesTransformer(input_dim=input_dim, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

# 데이터셋 및 데이터로더 생성
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 학습
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    val_loss = 0.0

    # 학습
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), -1, input_dim)  # 입력 차원 조정
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    # 검증
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.view(inputs.size(0), -1, input_dim)  # 입력 차원 조정
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

# 모델 평가
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(inputs.size(0), -1, input_dim)  # 입력 차원 조정
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# 성능 평가
print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred,digits=4))

# F1 스코어 계산
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 Score (weighted): {f1:.4f}')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/1, Train Loss: 1.3458, Validation Loss: 1.1611
Accuracy: 0.43376
              precision    recall  f1-score   support

           0     0.9817    0.2780    0.4333     29650
           1     0.9391    0.2867    0.4393     30079
           2     0.4837    0.5459    0.5129     29969
           3     0.3007    0.7423    0.4280     30220
           4     0.3920    0.3126    0.3478     30082

    accuracy                         0.4338    150000
   macro avg     0.6194    0.4331    0.4323    150000
weighted avg     0.6182    0.4338    0.4322    150000

F1 Score (weighted): 0.4322


In [ ]:
#ConvLSTM
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.signal import stft
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn


# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ConvLSTM 모델 정의
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_classes, bias=True):
        super(ConvLSTM, self).__init__()

        self.conv_lstm_cell = ConvLSTMCell(input_dim, hidden_dim, kernel_size, bias)
        self.fc = nn.Linear(hidden_dim * input_dim, num_classes)  # hidden_dim * input_dim 크기 변경

    def forward(self, x):
        batch_size, seq_len, height, width = x.size()
        h, c = self.conv_lstm_cell.init_hidden(batch_size, (height, width))

        for t in range(seq_len):
            h, c = self.conv_lstm_cell(x[:, t, :, :].unsqueeze(1), (h, c))

        h = h.view(batch_size, -1)
        out = self.fc(h)

        return out

# 하이퍼파라미터 설정
input_dim = 1  # 입력 채널 수 (임베딩 차원을 채널로 사용)
hidden_dim = 64
kernel_size = 3
num_classes = len(np.unique(y))

# 모델 초기화
conv_lstm_model = ConvLSTM(input_dim, hidden_dim, kernel_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv_lstm_model.parameters(), lr=0.001)

# 데이터셋 및 데이터로더 생성
# 임베딩을 (batch_size, seq_len, height, width) 형태로 변환
X_train_reshaped = torch.tensor(X_train, dtype=torch.float32).unsqueeze(2).unsqueeze(3)
X_test_reshaped = torch.tensor(X_test, dtype=torch.float32).unsqueeze(2).unsqueeze(3)
train_dataset = TensorDataset(X_train_reshaped, torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(X_test_reshaped, torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 학습
num_epochs = 1
conv_lstm_model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = conv_lstm_model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# 모델 평가
conv_lstm_model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = conv_lstm_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# 성능 평가
print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred,digits=4))

# F1 스코어 계산
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 Score (weighted): {f1:.4f}')


Epoch 1/1, Loss: 0.6155
Accuracy: 0.73642
              precision    recall  f1-score   support

           0     0.6414    0.6422    0.6418     59643
           1     0.7336    0.6888    0.7105     59988
           2     0.9820    0.9194    0.9497     60001
           3     0.5905    0.6576    0.6222     60196
           4     0.7663    0.7738    0.7701     60172

    accuracy                         0.7364    300000
   macro avg     0.7428    0.7363    0.7388    300000
weighted avg     0.7428    0.7364    0.7389    300000

F1 Score (weighted): 0.7389


In [ ]:
#LSTM AE attention
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
import scipy.signal
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import LSTM, Layer, Dense, Dropout, RepeatVector, TimeDistributed, Input, LeakyReLU
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import kneighbors_graph
import keras.backend as K


class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[-1],),
                                 initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# 데이터 파일 로드
normal_data = pd.read_csv(path+"300rpm normal data/stream2024_4_22_23_27.csv")
carriage_data = pd.read_csv(path+"300rpm carriage damage/stream2024_4_23_3_1.csv")
highspeed_data = pd.read_csv(path+"300rpm high-speed damage/stream2024_4_23_0_24.csv")
lack_data = pd.read_csv(path+"300rpm lack of lubrication/stream2024_4_23_2_10.csv")
corrosion_data = pd.read_csv(path+"300rpm oxidation and corrosion/stream2024_4_23_1_19.csv")

# 데이터 정규화 함수 정의
def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)
def concatenate_data(motor_data, sound_data):
    return np.hstack((motor_data, sound_data))

# 모터 및 소리 데이터 정규화
normal_motor_data_normalized = normalize_data(normal_data[['motor1_x', 'motor1_y', 'motor1_z']])
normal_sound_data_normalized = normalize_data(normal_data[['sound']])
carriage_motor_data_normalized = normalize_data(carriage_data[['motor1_x', 'motor1_y', 'motor1_z']])
carriage_sound_data_normalized = normalize_data(carriage_data[['sound']])
highspeed_motor_data_normalized = normalize_data(highspeed_data[['motor1_x', 'motor1_y', 'motor1_z']])
highspeed_sound_data_normalized = normalize_data(highspeed_data[['sound']])
lack_motor_data_normalized = normalize_data(lack_data[['motor1_x', 'motor1_y', 'motor1_z']])
lack_sound_data_normalized = normalize_data(lack_data[['sound']])
corrosion_motor_data_normalized = normalize_data(corrosion_data[['motor1_x', 'motor1_y', 'motor1_z']])
corrosion_sound_data_normalized = normalize_data(corrosion_data[['sound']])


# 레이블 생성
y_normal = np.zeros(X_normal.shape[0])
y_carriage = np.ones(X_carriage.shape[0])
y_highspeed = np.full((X_highspeed.shape[0],), 2)
y_lack = np.full((X_lack.shape[0],), 3)
y_corrosion = np.full((X_corrosion.shape[0],), 4)

# 데이터셋 합치기
X = np.concatenate((X_normal, X_carriage, X_highspeed, X_lack, X_corrosion), axis=0)
y = np.concatenate((y_normal, y_carriage, y_highspeed, y_lack, y_corrosion), axis=0)
# 레이블을 원-핫 인코딩으로 변환
y = to_categorical(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 형태 확인
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# 데이터 형태 변환
X_train = X_train.reshape(X_train.shape[0],-1, X_train.shape[1])  # (샘플 수, 시간, 특징 수)
X_test = X_test.reshape(X_test.shape[0], -1,X_test.shape[1])
print("Train data shape after reshape:", -1,X_train.shape)
print("Test data shape after reshape:", -1,X_test.shape)

# LSTM Autoencoder 모델 정의 (Attention 추가)
def create_lstm_ae_with_attention(input_shape):
    inputs = Input(shape=input_shape)
    encoded = LSTM(64, return_sequences=True)(inputs)
    encoded = LSTM(32, return_sequences=True)(encoded)
    attention = Attention()(encoded)
    repeated = RepeatVector(input_shape[0])(attention)
    decoded = LSTM(32, return_sequences=True)(repeated)
    decoded = LSTM(64, return_sequences=True)(decoded)
    outputs = TimeDistributed(Dense(input_shape[1]))(decoded)
    model = Model(inputs, outputs)
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # (시간, 특징 수)
model = create_lstm_ae_with_attention(input_shape)

model.compile(optimizer='adam', loss='mse')

# 얼리스탑 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 모델 훈련
print("Training LSTM Autoencoder model...")
history = model.fit(X_train, X_train, epochs=1, batch_size=256, validation_data=(X_test, X_test), callbacks=[early_stopping])


# LSTM Autoencoder에서 인코딩된 특징 추출
encoder = Model(inputs=model.input, outputs=model.layers[3].output)

X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)



# 다중 클래스 분류 모델 정의
classifier = Sequential()
classifier.add(Dense(64, input_shape=(X_train_encoded.shape[1],)))
classifier.add(LeakyReLU(alpha=0.01))  # Leaky ReLU 활성화 함수 사용
classifier.add(Dropout(0.5))
classifier.add(Dense(32))
classifier.add(LeakyReLU(alpha=0.01))  # Leaky ReLU 활성화 함수 사용
classifier.add(Dropout(0.2))
classifier.add(Dense(y_train.shape[1], activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 다중 클래스 분류 모델 훈련
print("Training classifier model...")
history = classifier.fit(X_train_encoded, y_train, epochs=1, batch_size=256, validation_data=(X_test_encoded, y_test), callbacks=[early_stopping])

# 분류 모델 서머리


# 테스트 데이터에 대한 예측 및 평가
y_test_pred = classifier.predict(X_test_encoded)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

print("Accuracy on test set:", accuracy_score(y_test_true_classes, y_test_pred_classes))
print(classification_report(y_test_true_classes, y_test_pred_classes, target_names=['Normal', 'Carriage Damage', 'High-Speed Damage', 'Lack of Lubrication', 'Oxidation and Corrosion'],digits=4))

Train data shape: (1200000, 4)
Test data shape: (300000, 4)
Train data shape after reshape: -1 (1200000, 1, 4)
Test data shape after reshape: -1 (300000, 1, 4)
Training LSTM Autoencoder model...
9375/9375 [==============================] - 22s 2ms/step
Training classifier model...
9375/9375 [==============================] - 15s 2ms/step
Accuracy on test set: 0.29678333333333334
                         precision    recall  f1-score   support

                 Normal     0.3019    0.4335    0.3559     59643
        Carriage Damage     0.2323    0.0550    0.0889     59988
      High-Speed Damage     0.5315    0.2688    0.3570     60001
    Lack of Lubrication     0.2316    0.3297    0.2720     60196
Oxidation and Corrosion     0.2842    0.3974    0.3314     60172

               accuracy                         0.2968    300000
              macro avg     0.3163    0.2969    0.2811    300000
           weighted avg     0.3163    0.2968    0.2810    300000

